<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Personalized_Recommendations_with_a_Transformer_Based_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Sample dataset
class RecommendationDataset(Dataset):
    def __init__(self, user_ids, item_ids, ratings):
        self.user_ids = user_ids
        self.item_ids = item_ids
        self.ratings = ratings

    def __len__(self):
        return len(self.user_ids)

    def __getitem__(self, idx):
        return self.user_ids[idx], self.item_ids[idx], self.ratings[idx]

# Neural Collaborative Filtering Model
class NCF(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim):
        super(NCF, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim * 2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, user_ids, item_ids):
        user_embed = self.user_embedding(user_ids)
        item_embed = self.item_embedding(item_ids)
        x = torch.cat([user_embed, item_embed], dim=-1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Create dummy data
user_ids = torch.tensor([0, 1, 2, 0, 1, 2], dtype=torch.long)
item_ids = torch.tensor([0, 1, 2, 1, 2, 0], dtype=torch.long)
ratings = torch.tensor([5, 4, 3, 4, 5, 2], dtype=torch.float)

dataset = RecommendationDataset(user_ids, item_ids, ratings)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Model and training setup
num_users = 3
num_items = 3
embedding_dim = 8
model = NCF(num_users, num_items, embedding_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for user_ids, item_ids, ratings in dataloader:
        optimizer.zero_grad()
        outputs = model(user_ids, item_ids)
        loss = criterion(outputs.squeeze(), ratings)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item():.4f}')

# Making a recommendation for a user
user_id = torch.tensor([0], dtype=torch.long)
item_id = torch.tensor([1], dtype=torch.long)
model.eval()
with torch.no_grad():
    recommendation_score = model(user_id, item_id).item()
    print(f"Recommendation score for user {user_id.item()} and item {item_id.item()}: {recommendation_score:.4f}")